In [94]:
from datetime import datetime
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [95]:

model=load_model('Models\churn_model.h5')

with open('Models\churn_scaler.pkl', 'rb') as file:
     scaler = pickle.load(file)

In [96]:
# Example input input_df
data=pd.read_excel(r'data\CS_Main.xlsx')
data

,transaction_id,customer_id,product_id,product_name,category,purchase_date,quantity,price_per_unit,total_amount,payment_method,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,181,181,P002,Tennis Racket,Rackets,2025-01-11 19:20:52,1,3500,3500,Debit Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,186,186,P009,Running Shorts,Apparel,2025-01-16 12:22:58,1,800,800,UPI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,129,129,P009,Running Shorts,Apparel,2025-02-19 23:52:00,3,800,2400,Credit Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,181,181,P009,Running Shorts,Apparel,2025-05-01 19:04:58,2,800,1600,Cash,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,197,197,P003,Basketball,Balls,2025-02-19 08:21:24,3,1200,3600,UPI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,188,188,P002,Tennis Racket,Rackets,2025-04-09 05:45:33,2,3500,7000,UPI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,137,137,P008,Cricket Bat,Bats,2024-12-14 16:26:06,2,5000,10000,UPI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,104,104,P003,Basketball,Balls,2025-05-01 07:27:53,1,1200,1200,Debit Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,172,172,P006,Tennis Balls Pack,Balls,2025-03-11 20:40:23,3,500,1500,Debit Card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# # One-hot encode 'Geography'
# geo_encoded = label_encoder_geo.transform([[input_input_df['Geography']]]).toarray()
# geo_encoded_df = pd.input_dfFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
# geo_encoded_df


In [98]:
## Encode categorical variables
# input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
# input_df

In [99]:
# ## concatination one hot encoded 
# input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
# input_df

In [100]:
group = data.groupby('customer_id').agg(
    Monetary=('total_amount', 'sum'),
    total_quantity=('quantity', 'sum'),
    Frequency=('transaction_id', 'count'),
    num_unique_products=('product_id', 'nunique'),
    last_purchase_date=('purchase_date', 'max'),
    avg_price_per_unit=('price_per_unit', 'mean'),
    store_visit_frequency=('purchase_date', 'nunique'),
).reset_index()
group

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency
0,100,50100,34,19,9,2025-05-13 09:55:23,1468.421053,19
1,101,117000,45,21,7,2025-05-21 01:59:44,2495.238095,21
2,102,71800,42,22,9,2025-05-28 00:24:42,1759.090909,22
3,103,137500,62,28,10,2025-05-23 05:33:37,2082.142857,28
4,104,95100,55,27,9,2025-05-15 06:44:49,1785.185185,27
...,...,...,...,...,...,...,...,...
96,196,53600,25,14,9,2025-05-23 06:36:39,2057.142857,14
97,197,54800,38,21,10,2025-05-25 11:18:04,1595.238095,21
98,198,55800,33,15,7,2025-05-27 04:28:10,1780.000000,15
99,199,21800,25,12,7,2025-04-24 17:03:45,1150.000000,12


In [ ]:
membership_start = data.groupby('customer_id')['purchase_date'].min().reset_index()
membership_start.rename(columns={'purchase_date':'membership_start_date'}, inplace=True)

data = group.merge(membership_start, on='customer_id', how='left')
data


,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,membership_start_date
0,100,50100,34,19,9,2025-05-13 09:55:23,1468.421053,19,2024-12-11 03:37:40
1,101,117000,45,21,7,2025-05-21 01:59:44,2495.238095,21,2024-11-30 09:07:33
2,102,71800,42,22,9,2025-05-28 00:24:42,1759.090909,22,2024-12-05 14:18:41
3,103,137500,62,28,10,2025-05-23 05:33:37,2082.142857,28,2024-12-13 18:51:29
4,104,95100,55,27,9,2025-05-15 06:44:49,1785.185185,27,2024-12-06 21:00:40
...,...,...,...,...,...,...,...,...,...
96,196,53600,25,14,9,2025-05-23 06:36:39,2057.142857,14,2024-12-23 14:53:07
97,197,54800,38,21,10,2025-05-25 11:18:04,1595.238095,21,2024-11-29 17:34:22
98,198,55800,33,15,7,2025-05-27 04:28:10,1780.000000,15,2024-12-02 02:52:59
99,199,21800,25,12,7,2025-04-24 17:03:45,1150.000000,12,2024-12-04 21:31:35


In [102]:
reference_date = pd.to_datetime(data['last_purchase_date'].max())
today = pd.to_datetime(datetime.today().date())



data['Active_days'] = ((reference_date - data['membership_start_date']).dt.days).round().astype(int)

data['Avg_purchase_gap_days'] = data.apply(
    lambda x: x['Active_days'] / x['store_visit_frequency'] if x['store_visit_frequency'] > 0 else x['Active_days'],
    axis=1
)


data['average_purchase_value'] = data['Monetary'] / data['total_quantity']
data['Recency'] = (today - data['last_purchase_date']).dt.days

data['last_purchase_date'] = data['last_purchase_date'].dt.date
data['membership_start_date'] = data['membership_start_date'].dt.date

data


,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,membership_start_date,Active_days,Avg_purchase_gap_days,average_purchase_value,Recency
0,100,50100,34,19,9,2025-05-13,1468.421053,19,2024-12-11,168,8.842105,1473.529412,23
1,101,117000,45,21,7,2025-05-21,2495.238095,21,2024-11-30,179,8.523810,2600.000000,15
2,102,71800,42,22,9,2025-05-28,1759.090909,22,2024-12-05,173,7.863636,1709.523810,8
3,103,137500,62,28,10,2025-05-23,2082.142857,28,2024-12-13,165,5.892857,2217.741935,13
4,104,95100,55,27,9,2025-05-15,1785.185185,27,2024-12-06,172,6.370370,1729.090909,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,196,53600,25,14,9,2025-05-23,2057.142857,14,2024-12-23,155,11.071429,2144.000000,13
97,197,54800,38,21,10,2025-05-25,1595.238095,21,2024-11-29,179,8.523810,1442.105263,11
98,198,55800,33,15,7,2025-05-27,1780.000000,15,2024-12-02,177,11.800000,1690.909091,9
99,199,21800,25,12,7,2025-04-24,1150.000000,12,2024-12-04,174,14.500000,872.000000,42


In [106]:
input=input[['Monetary','Frequency','Avg_purchase_gap_days','Recency']]      
input

,Monetary,Frequency,Avg_purchase_gap_days,Recency
0,50100,19,8.842105,23
1,117000,21,8.523810,15
2,71800,22,7.863636,8
3,137500,28,5.892857,13
4,95100,27,6.370370,21
...,...,...,...,...
96,53600,14,11.071429,13
97,54800,21,8.523810,11
98,55800,15,11.800000,9
99,21800,12,14.500000,42


In [107]:
## Scaling the input input_df
input_scaled=scaler.transform(input)
input_scaled

array([[-1.31366591e+00, -5.21252511e-01,  2.73738435e-01,
         2.77350098e-01],
       [ 1.66449971e+00,  4.00963470e-02,  3.79583510e-02,
        -4.28631970e-01],
       [-3.47654040e-01,  3.20770776e-01, -4.51070133e-01,
        -1.04636628e+00],
       [ 2.57709156e+00,  2.00481735e+00, -1.91094041e+00,
        -6.05127487e-01],
       [ 6.89584508e-01,  1.72414292e+00, -1.55721872e+00,
         1.00854581e-01],
       [ 1.69566138e+00, -1.08260137e+00,  7.82866156e-01,
         1.26068226e-02],
       [-6.05850760e-01,  4.00963470e-02, -1.73686763e-01,
        -5.16879728e-01],
       [ 1.59836065e-01, -1.08260137e+00,  8.56822494e-02,
        -6.05127487e-01],
       [-8.64047480e-01,  3.20770776e-01, -3.16386878e-01,
        -8.69870762e-01],
       [-8.06175802e-01, -1.92462466e+00,  2.77170775e+00,
         8.06836649e-01],
       [-8.46240810e-01, -1.08260137e+00,  1.08788411e+00,
        -6.93375245e-01],
       [ 9.56684563e-01,  1.72414292e+00, -2.07849279e+00,
      

In [108]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[8.69358599e-01],
       [4.03874263e-04],
       [2.28085201e-02],
       [2.86436030e-08],
       [3.96336836e-05],
       [2.92104227e-03],
       [1.89607069e-01],
       [7.52470866e-02],
       [8.13309029e-02],
       [9.68483150e-01],
       [5.55341780e-01],
       [1.91553977e-06],
       [3.73682112e-01],
       [5.28906047e-01],
       [1.06656671e-05],
       [4.60124254e-04],
       [6.54596925e-01],
       [2.04348427e-04],
       [4.15944401e-03],
       [9.49705660e-01],
       [8.16263020e-01],
       [6.84060931e-01],
       [4.68569378e-05],
       [7.66236544e-01],
       [8.18474352e-01],
       [8.03152740e-01],
       [9.96046305e-01],
       [3.96733433e-01],
       [1.95202930e-03],
       [1.89938361e-03],
       [9.59965229e-01],
       [3.49971280e-03],
       [9.99641597e-01],
       [3.99013443e-05],
       [4.51499224e-01],
       [3.42675025e-06],
       [7.33168840e-01],
       [2.06162542e-01],
       [6.82645012e-03],
       [6.78114546e-03],


In [109]:
input['prediction_proba'] = prediction.flatten()
input

,Monetary,Frequency,Avg_purchase_gap_days,Recency,prediction_proba
0,50100,19,8.842105,23,8.693586e-01
1,117000,21,8.523810,15,4.038743e-04
2,71800,22,7.863636,8,2.280852e-02
3,137500,28,5.892857,13,2.864360e-08
4,95100,27,6.370370,21,3.963368e-05
...,...,...,...,...,...
96,53600,14,11.071429,13,7.690980e-01
97,54800,21,8.523810,11,2.712004e-01
98,55800,15,11.800000,9,4.999343e-01
99,21800,12,14.500000,42,9.996204e-01


In [111]:

input['risk_level'] = input['prediction_proba'].apply(
    lambda p: 'High' if p > 0.7 else 'Medium' if p > 0.3 else 'Low'
)

churn_labels = (prediction[:, 0] > 0.5).astype(int)  # 1 = churn, 0 = not churn

input['churn_prediction'] = churn_labels

input


,Monetary,Frequency,Avg_purchase_gap_days,Recency,prediction_proba,risk_level,churn_prediction
0,50100,19,8.842105,23,8.693586e-01,High,1
1,117000,21,8.523810,15,4.038743e-04,Low,0
2,71800,22,7.863636,8,2.280852e-02,Low,0
3,137500,28,5.892857,13,2.864360e-08,Low,0
4,95100,27,6.370370,21,3.963368e-05,Low,0
...,...,...,...,...,...,...,...
96,53600,14,11.071429,13,7.690980e-01,High,1
97,54800,21,8.523810,11,2.712004e-01,Low,0
98,55800,15,11.800000,9,4.999343e-01,Medium,0
99,21800,12,14.500000,42,9.996204e-01,High,1


In [ ]:
input=input.drop
input_df

NameError: name 'input_df' is not defined

In [81]:
# for i, pred in enumerate(prediction):
#     prediction_proba = pred[0]
#     if prediction_proba > 0.5:
#         print(f'Customer {i} is likely to churn.')
#     else:
#         print(f'Customer {i} is not likely to churn.')


In [137]:
churn_status = []

for i, prediction_proba in enumerate(churn_probs):
    if prediction_proba > 0.5:
        churn_status.append(f'Customer {i} is likely to churn.')
    else:
        churn_status.append(f'Customer {i} is not likely to churn.')

input_df['prediction'] = churn_status

input_df

,customer_id,age,gender,location,membership_start_date,average_purchase_value,purchase_frequency_per_month,last_purchase_date,product_preference,preferred_channel,store_visit_frequency,days_since_last_visit,product_segment,Active_days,Avg_purchase_gap_days,prediction_proba,churn_prediction,prediction
0,CUST1000,58,Male,East Jessetown,30-08-2020,375.94,2.6,21-10-2023,Gym Gear,In-Store,17,108,Casual Gym-Goer,1715,100.882353,2.012895e-22,0,Customer 0 is not likely to churn.
1,CUST1001,20,Male,Herrerafurt,21-01-2025,124.95,5.2,18-02-2025,Running Shoes,In-Store,20,107,Casual Gym-Goer,110,5.500000,1.000000e+00,1,Customer 1 is likely to churn.
2,CUST1002,32,Female,Shawhaven,06-01-2024,153.53,8.7,13-04-2024,Gym Gear,Online,10,55,General Buyer,491,49.100000,1.825968e-03,0,Customer 2 is not likely to churn.
3,CUST1003,39,Male,Herrerafurt,17-09-2020,202.37,3.7,02-05-2023,Supplements,Online,1,137,Health Conscious Buyer,1697,1697.000000,0.000000e+00,0,Customer 3 is not likely to churn.
4,CUST1004,25,Female,Cassandraton,17-06-2023,284.98,8.3,24-02-2025,Training Wear,Online,18,17,Casual Gym-Goer,694,38.555556,1.092046e-11,0,Customer 4 is not likely to churn.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,CUST1995,62,Male,North Susan,11-01-2025,173.67,5.1,12-04-2025,Running Shoes,Online,18,55,Casual Gym-Goer,120,6.666667,1.000000e+00,1,Customer 995 is likely to churn.
996,CUST1996,49,Female,Lake Lisaport,06-12-2024,483.92,0.8,10-05-2025,Supplements,In-Store,6,57,Fitness Enthusiast,156,26.000000,9.999867e-01,1,Customer 996 is likely to churn.
997,CUST1997,62,Other,Lisatown,20-03-2022,463.12,7.8,08-04-2023,Yoga Accessories,Online,9,168,General Buyer,1148,127.555556,2.358130e-10,0,Customer 997 is not likely to churn.
998,CUST1998,28,Male,Saramouth,20-01-2021,208.30,4.3,07-11-2024,Yoga Accessories,Online,12,135,General Buyer,1572,131.000000,4.600708e-24,0,Customer 998 is not likely to churn.


In [112]:
#prediction_proba = prediction

In [113]:
#prediction_proba

array([[2.01289485e-22],
       [1.00000000e+00],
       [1.82596827e-03],
       [0.00000000e+00],
       [1.09204598e-11],
       [5.89913160e-19],
       [5.01606395e-20],
       [5.85595176e-33],
       [0.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [9.99919653e-01],
       [0.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [9.99815106e-01],
       [1.00000000e+00],
       [1.91436315e-35],
       [7.13145077e-01],
       [1.16360494e-24],
       [1.08784552e-28],
       [8.07170273e-21],
       [1.00000000e+00],
       [2.45419751e-09],
       [9.99999881e-01],
       [1.71917986e-30],
       [3.86390503e-35],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.54735166e-18],
       [3.02911725e-33],
       [1.00000000e+00],
       [0.00000000e+00],
       [1.31316035e-14],
       [1.14857979e-09],
       [9.48077142e-01],
       [1.16717978e-08],
       [0.00000000e+00],
       [6.17144234e-35],


In [107]:
# if prediction_proba > 0.5:
#     print('The customer is likely to churn.')
# else:
#     print('The customer is not likely to churn.')